In [ ]:
import os
import random
from tqdm import tqdm
from sortedcontainers import SortedList


import numpy as np

import pretty_midi
import note_seq
from note_seq.midi_io import midi_to_note_sequence, note_sequence_to_pretty_midi

import IPython.display

from helpers import *

In [ ]:
def preprocess(notes):
    text = "PIECE_START"

    sorted_lists = {}

    for note in notes:
        current_instrument = note.instrument
        current_note = note.pitch
        current_start_time = note.start_time
        current_end_time = note.end_time
        current_bar = current_start_time//2

        sorted_list = sorted_lists.setdefault(current_instrument, SortedList())

        sorted_list.add((current_bar, current_start_time, f'NOTE_ON={current_note}'))
        sorted_list.add((current_bar, current_end_time, f'NOTE_OFF={current_note}'))    

    for instrument, sorted_list in sorted_lists.items():
        # TODO fill density
        text += f' TRACK_START INST={instrument} DENSITY=0 BAR_START'
        time_slider = 0    
        current_bar = 0

        for el in sorted_list:
            event_bar, event_time, event = el

            while event_bar != current_bar:
                current_bar = current_bar + 1
                text += ' BAR_END BAR_START'
                time_slider = current_bar*2

            delta_value = (event_time - time_slider)*8.0

            if delta_value > 0:
                text += f' TIME_DELTA={delta_value}'

            text += f' {event}'
            time_slider = event_time

        text += f' BAR_END TRACK_END'
    text += ' PIECE_END\n'
    return text

## Test case

In [ ]:
def case1():
    # simple note shifted 1sec to right
    original = 'PIECE_START TRACK_START INST=0 DENSITY=0 BAR_START TIME_DELTA=8.0 NOTE_ON=69 TIME_DELTA=8.0 NOTE_OFF=69 BAR_END TRACK_END PIECE_END\n'
    original_note_sequence = token_sequence_to_note_sequence(original, use_program=False)
    
    reencoded = preprocess(original_note_sequence.notes)
    
    assert original == reencoded
    

def case2():
    # multiple simultanuous notes shifted 1 sec
    original = 'PIECE_START TRACK_START INST=1 DENSITY=0 BAR_START NOTE_ON=50 TIME_DELTA=8.0 NOTE_ON=53 TIME_DELTA=8.0 NOTE_OFF=50 TIME_DELTA=8.0 NOTE_OFF=53 BAR_END BAR_START NOTE_ON=57 TIME_DELTA=8.0 NOTE_ON=48 TIME_DELTA=8.0 NOTE_OFF=57 TIME_DELTA=8.0 NOTE_OFF=48 BAR_END TRACK_END PIECE_END\n'
    original_note_sequence = token_sequence_to_note_sequence(original, use_program=False)
    
    reencoded = preprocess(original_note_sequence.notes)
    
    assert original == reencoded
    
def case3():
    # one note is played while the other one is being played (starts later, ends earlier)
    original = 'PIECE_START TRACK_START INST=0 DENSITY=0 BAR_START TIME_DELTA=8.0 NOTE_ON=69 TIME_DELTA=2.0 NOTE_ON=59 TIME_DELTA=4.0 NOTE_OFF=59 TIME_DELTA=2.0 NOTE_OFF=69 BAR_END TRACK_END PIECE_END\n'
    original_note_sequence = token_sequence_to_note_sequence(original, use_program=False)
    
    reencoded = preprocess(original_note_sequence.notes)
    
    assert original == reencoded
    
    
def case4():
    # the piece starts from 4th bar
    original = 'PIECE_START TRACK_START INST=0 DENSITY=0 BAR_START BAR_END BAR_START BAR_END BAR_START TIME_DELTA=8.0 NOTE_ON=69 TIME_DELTA=2.0 NOTE_ON=59 TIME_DELTA=4.0 NOTE_OFF=59 TIME_DELTA=2.0 NOTE_OFF=69 BAR_END TRACK_END PIECE_END\n'
    original_note_sequence = token_sequence_to_note_sequence(original, use_program=False)
    
    reencoded = preprocess(original_note_sequence.notes)
    
    assert original == reencoded
    
    
def case5():
    # Same case 4 with two tracks
    original = 'PIECE_START TRACK_START INST=0 DENSITY=0 BAR_START BAR_END BAR_START BAR_END BAR_START TIME_DELTA=8.0 NOTE_ON=69 TIME_DELTA=2.0 NOTE_ON=59 TIME_DELTA=4.0 NOTE_OFF=59 TIME_DELTA=2.0 NOTE_OFF=69 BAR_END TRACK_END TRACK_START INST=1 DENSITY=0 BAR_START BAR_END BAR_START BAR_END BAR_START TIME_DELTA=8.0 NOTE_ON=69 TIME_DELTA=2.0 NOTE_ON=59 TIME_DELTA=4.0 NOTE_OFF=59 TIME_DELTA=2.0 NOTE_OFF=69 BAR_END TRACK_END PIECE_END\n'
    original_note_sequence = token_sequence_to_note_sequence(original, use_program=False)
    
    reencoded = preprocess(original_note_sequence.notes)
    
    assert original == reencoded
    

In [ ]:
case1()
case2()
case3()
case4()
case5()

## On real examples

### Love the way you lie

In [ ]:
midi_data = pretty_midi.PrettyMIDI('LovetheWayYouLie.mid')
note_sequence = midi_to_note_sequence(midi_data)


In [ ]:
note_seq.plot_sequence(note_sequence)
note_seq.play_sequence(note_sequence)


In [ ]:
encoding = preprocess(note_sequence.notes)

In [ ]:
sequence_new = token_sequence_to_note_sequence(encoding, use_program=False)
note_seq.plot_sequence(sequence_new)
note_seq.play_sequence(sequence_new)

### stan

In [ ]:
midi_data = pretty_midi.PrettyMIDI('eminemstanexc.mid')
note_sequence = midi_to_note_sequence(midi_data)
note_seq.plot_sequence(note_sequence)
note_seq.play_sequence(note_sequence)


In [ ]:
encoding = preprocess(note_sequence.notes)

In [ ]:
sequence_new = token_sequence_to_note_sequence(encoding, use_program=False)
note_seq.plot_sequence(sequence_new)
note_seq.play_sequence(sequence_new)

### Mockingbird

In [ ]:
midi_data = pretty_midi.PrettyMIDI('EminemMockingbird.mid')
note_sequence = midi_to_note_sequence(midi_data)
note_seq.plot_sequence(note_sequence)
note_seq.play_sequence(note_sequence)


In [ ]:
encoding = preprocess(note_sequence.notes)

In [ ]:
sequence_new = token_sequence_to_note_sequence(encoding, use_program=False)
note_seq.plot_sequence(sequence_new)
note_seq.play_sequence(sequence_new)